# Example of analysis

Import the necessary libraries

In [1]:
import bcells
from bcells.data.fetchers import get_example_traces
from bcells.processing.plotting import plot_traces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

The package contains a few trace examples on which we will perform the analysis in this example notebook. We remark that the plots are thus not very insightful since very few data is actually used to generate these plots. They are created for the purposes of this example and not because the describe some scientific phenomenon well. We start by loading the data and plotting the traces.

In [2]:
example_traces = get_example_traces()

In [ ]:
fig, axs = plt.subplots(1, len(example_traces), figsize=(10, 5))
for i in range(axs.size):
    dat = example_traces[i]['traces'].to_numpy()
    conc = example_traces[i]['conc']
    axs[i].plot(np.arange(dat.shape[1]) / 2, dat.T, linewidth=0.5)
    axs[i].set_xlabel("Time (s)")
    axs[i].set_title("Example traces for {} muM".format(conc))
plt.show()

Now, we call the analysis function which calculates a standardization of the traces, certain characerteristics of the traces and a quick approximate barycenter.

In [4]:
for i in range(len(example_traces)):
    traces_stand, characs, app_bary, charac_bary = bcells.processing.pipeline.process_traces(example_traces[i]['traces'])
    example_traces[i]['traces_stand'] = traces_stand
    example_traces[i]['charac'] = characs
    example_traces[i]['app_bary'] = app_bary
    example_traces[i]['charac_bary'] = charac_bary
    log_0_val_plot = 5e-2
    if example_traces[i]['conc'] == 0:
        example_traces[i]['charac']['conc_plot'] = log_0_val_plot
        example_traces[i]['charac_bary']['conc_plot'] = log_0_val_plot
    else:
        example_traces[i]['charac']['conc_plot'] = example_traces[i]['conc']
        example_traces[i]['charac_bary']['conc_plot'] = example_traces[i]['conc']
charac_df_all = pd.concat([elem['charac'] for elem in example_traces], ignore_index=True)
charac_barys = pd.concat([elem['charac_bary'] for elem in example_traces], ignore_index=True)

Now, we plot the standardized traces and the barycenter as an example.

In [ ]:
fig, axs = plt.subplots(1, len(example_traces), figsize=(10, 5))
for i in range(axs.size):
    dat = example_traces[i]['traces_stand']
    conc = example_traces[i]['conc']
    axs[i] = plot_traces(dat, conc=conc, ax=axs[i])
plt.show()

In [ ]:
from bcells.processing.plotting import plot_traces

fig, axs = plt.subplots(1, len(example_traces), figsize=(10, 5))
for i in range(axs.size):
    dat = example_traces[i]['traces_stand']
    conc = example_traces[i]['conc']
    bary = bcells.processing.standardization.divide_by_abs_mean_before_light(example_traces[i]['app_bary'])
    axs[i] = plot_traces(dat, conc=conc, bary=bary, ax=axs[i])
plt.show()

Furthermore, we can plot the characteristics of the traces and the characteristics of the barycenter.

In [11]:
characteristics = []
for elem in charac_df_all.columns.values.tolist():
    if elem != "conc_plot":
        characteristics.append(elem)

In [ ]:
concs_true = [elem['conc'] for elem in example_traces]
concs_plot = [elem['charac']['conc_plot'].iloc[0] for elem in example_traces]
fig, ax = plt.subplots(1, 1, figsize=(11, 5))
ax = bcells.processing.plotting.plot_all_charac(dat=charac_df_all, y='peak_time', concs_true=concs_true, concs_plot=concs_plot, ax=ax)
plt.show()

The following is a scatter plot since we have only on characteristics value for each barycenter.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax = bcells.processing.plotting.plot_charac_barys(dat=charac_barys, y='peak_time', concs_true=concs_true, concs_plot=concs_plot, ax=ax)
plt.show()